# PDFから英単語(TOEIC公開テスト出題)を抽出し、APIで意味を調べ、MySQLのテーブルにデータを追加する

In [ ]:
# ライブラリをインポート
import pdfplumber
import re
from pprint import pprint
import requests
import mysql.connector
import os

In [ ]:
# PDFファイル
pdf_file = "toeic_vocabulary.pdf"

In [ ]:
# PDFファイルの読み込み
with pdfplumber.open(pdf_file) as pdf:
    # 全ページの取得
    pages = pdf.pages

    # 全ページのテキストを抽出
    all_text = ""
    for page in pages:
        all_text += page.extract_text()

In [ ]:
# ページ数を確認
print("Number of pages:", len(pages))

Number of pages: 1


In [ ]:
# 全テキストを出力
print(all_text)

■2024年の公開テストで実際に出題された単語
・pursue
・experienced
・utensil
・subsidiary
・launch
・verify
・renovate
・satisfactory
・soar
・rigorous


In [ ]:
# 英単語を抽出する関数
def extract_words(text):
    words = re.findall(r"[a-zA-Z]+", text)
    return words

In [ ]:
# テキストから英単語を抽出
words_list = extract_words(all_text)
# 重複を除いてユニークな値のみをリストに追加
words_list_unique_value = list(set(words_list))

In [ ]:
# リストを確認
print(words_list_unique_value)

['pursue', 'experienced', 'satisfactory', 'soar', 'utensil', 'subsidiary', 'rigorous', 'verify', 'launch', 'renovate']


In [ ]:
# 英単語と意味を格納するリスト
words_and_meanings = []

In [ ]:
# 英単語と意味のタプルをリストに追加
for word in words_list_unique_value:
    # URL(APIのエンドポイント)
    url = f"https://api.dictionaryapi.dev/api/v2/entries/en/{word}"
    # データを取得
    response = requests.get(url)
    json_data = response.json()
    # 意味を取得しリストに追加
    meaning = json_data[0]["meanings"][0]["definitions"][0]["definition"]
    word_and_meaning_tuple = (word, meaning)
    words_and_meanings.append(word_and_meaning_tuple)

In [ ]:
# リストを確認
pprint(words_and_meanings)

[('pursue',
  'To follow urgently, originally with intent to capture or harm; to chase.'),
 ('experienced',
  'To observe certain events; undergo a certain feeling or process; or perform '
  "certain actions that may alter one or contribute to one's knowledge, "
  'opinions, or skills.'),
 ('satisfactory', 'Done to satisfaction; adequate or sufficient.'),
 ('soar', 'The act of soaring.'),
 ('utensil',
  'An instrument or device for domestic use, especially in the kitchen.'),
 ('subsidiary',
  'A company owned by a parent company or a holding company, also called '
  'daughter company or sister company.'),
 ('rigorous',
  'Showing, causing, or favoring rigour; scrupulously accurate or strict; '
  'thorough.'),
 ('verify', 'To substantiate or prove the truth of something'),
 ('launch',
  'The movement of a vessel from land into the water; especially, the sliding '
  'on ways from the stocks on which it is built. (Compare: to splash a ship.)'),
 ('renovate', 'To renew; to revamp something

In [ ]:
# ホスト
host = "localhost"

# ユーザー名 (環境変数から取得)
user = os.getenv("YOUR_USER_NAME")

# パスワード (環境変数から取得)
password = os.getenv("YOUR_PASSWORD")

# データベース名
db = "your_database_name"

In [ ]:
# MySQLに接続
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=db
)

In [ ]:
# カーソルを取得
cursor = conn.cursor()

In [ ]:
# SQL(insert文)
base_sql = "INSERT INTO toeic (word, meaning) VALUES (%s, %s)"

# バリュー
values = words_and_meanings

In [ ]:
# SQLを実行
cursor.executemany(base_sql, values)

In [ ]:
# コミット
conn.commit()

In [ ]:
# 追加されたレコード数を確認
print(cursor.rowcount, "record(s) inserted.")

10 record(s) inserted.


In [ ]:
# 接続をクローズ
cursor.close()
conn.close()